In [1]:
from src.workflow import *
from src.datasets.tshark import TsharkData
from src.models.classification import RandomForest

dataset = TsharkData("Data/tshark/scenario1", 0.3, "multi")
model = RandomForest()

wf = WorkFlow(dataset, model)

In [4]:
import os
os.getcwd()

'/home/aaron/projecten/ml-ot'

In [8]:
path_data = "/home/aaron/projecten/ml-ot/multirun/2023-04-06/10-23-32/0/../TsharkData_supervised.pkl"
output_dir = "/home/aaron/projecten/ml-ot/multirun/2023-04-06/10-23-32/0"
import pickle

if "multirun" in output_dir and os.path.exists(path_data):
    with open(path_data, 'rb') as f:
        dataset = pickle.load(f)

In [14]:
dataset.data

,frame.len,label,packets_per_min,packets_per_sec,max_packets,min_packets,mean_packets,packets_per_ip.dst,SYN,ACK,FIN,PSH
0,74,0,2.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0
1,74,0,2.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0
2,74,0,4.0,2.0,2.0,0.0,1.000000,0.0,1.0,1.0,0.0,0.0
3,74,0,4.0,2.0,2.0,0.0,1.000000,0.0,1.0,1.0,0.0,0.0
4,66,0,6.0,4.0,4.0,0.0,2.000000,2.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5279,66,0,822.0,22.0,22.0,0.0,10.833333,8.0,0.0,1.0,1.0,0.0
5280,66,0,824.0,24.0,24.0,0.0,11.846154,6.0,0.0,1.0,1.0,0.0
5281,66,0,824.0,24.0,24.0,0.0,11.846154,6.0,0.0,1.0,1.0,0.0
5282,66,0,826.0,26.0,26.0,0.0,12.857143,10.0,0.0,1.0,0.0,0.0


In [3]:
import shap
shap.Explainer?

Init signature:
shap.Explainer(
    model,
    masker=None,
    link=CPUDispatcher(<function identity at 0x7fe5a1480310>),
    algorithm='auto',
    output_names=None,
    feature_names=None,
    linearize_link=True,
    seed=None,
    **kwargs,
)
Docstring:     
Uses Shapley values to explain any machine learning model or python function.

This is the primary explainer interface for the SHAP library. It takes any combination
of a model and masker and returns a callable subclass object that implements
the particular estimation algorithm that was chosen.
Init docstring:
Build a new explainer for the passed model.

Parameters
----------
model : object or function
    User supplied function or model object that takes a dataset of samples and
    computes the output of the model for those samples.

masker : function, numpy.array, pandas.DataFrame, tokenizer, None, or a list of these for each model input
    The function used to "mask" out hidden features of the form `masked_args = masker(*

In [2]:
wf.execute()

'class' column not present in dataset resorting to binary labels


at log warning


ValueError: HydraConfig was not set

In [ ]:
import pandas as pd

df = pd.read_csv('Data/nprobe/NF-UNSW-NB15.csv')
df.head()

In [ ]:
#remove duplicates
df.drop_duplicates(inplace=True)

#preprocessing protocol layer 4
ip_prot_map = {6:'TCP', 17:'UDP', 1:'ICMP', 2:'IGMP', 58:'IPv6-ICMP', 47:'GRE', 0:'HOPOPT'}
df['PROTOCOL'] = df['PROTOCOL'].map(ip_prot_map).fillna('other')
df = pd.get_dummies(df, columns=['PROTOCOL'])

#dropping some columns
df.drop(columns=['IPV4_SRC_ADDR','L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT'], inplace=True)

#rename columns
df.rename(columns={'Label':'label', 'Attack':'class'}, inplace=True)
df

In [ ]:
df['class'].unique()

* We drop the flow identifiers such as IDs, source/destination IP and ports, timestamps and start/end time.
* we utilise the min-max normalisation technique to scale all datasets’ values between 0 to 1
* Due to the extreme imbalance in all datasets’ binary-class and multi-class labels, we set a custom class weight parameter, using Equation 1. To reliably evaluate the datasets.
 1. Weight_class = #total samples/ (#classes * #class samples)
* we conduct five cross-validation splits and collect the average metrics such as accuracy, Area Under the Curve (AUC), F1 Score, Detection Rate (DR), False Alarm Rate (FAR) and time required in microseconds (µs) to predict a single test sample.


In [ ]:
from sklearn.model_selection import train_test_split
x, y = df.drop(columns=['label','class']), df['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,
                                                   random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols_scale = ['L7_PROTO','IN_BYTES','OUT_BYTES','IN_PKTS','OUT_PKTS','TCP_FLAGS',
             'FLOW_DURATION_MILLISECONDS']

scaler = MinMaxScaler()

scaler.fit(x_train[cols_scale])
x_train[cols_scale], x_test[cols_scale] = scaler.transform(x_train[cols_scale]), scaler.transform(x_test[cols_scale])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train, y_train)


In [ ]:
pred_train, pred_test = model.predict(x_train), model.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

cf_train = classification_report(y_train, pred_train)
cf_test = classification_report(y_test, pred_test)

In [ ]:
print(cf_train)
print(cf_test)

# Explainable AI


## Scenario 1

In [ ]:
#inladen data en getrained model
import pickle
with open("data.pkl","rb") as f:
    dataset = pickle.load(f)

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
import shap
import numpy as np

clf = model.model
explainer = shap.Explainer(clf, dataset.x_train)
values = explainer.shap_values(dataset.x_train)

### Normale traffiek
Belangrijkste inputs voor het model om normale traffiek te voorspellen staan in het **blauw**.\
Inputs voor het model om abnormale traffiek te voorspellen staan in het **rood**.

De belangrijkste inputs zijn hier

- Frame.len: hoeveelheid data verstuurd
- packets_per_min: aantal berichten in de afgelopen minuut

In [ ]:
#voorbeeld van normale trafiek
display(shap.force_plot(explainer.expected_value[1], values[1][10,:], dataset.x_train.iloc[10,:]))

### ARP scan
Eerst wordt er een ARP scan uitgevoerd om te bepalen welke devices op het netwerk zitten en wat hun bijbehorende IP adres is.\
Zo weet de hacker welke devices mogelijks een vulnerability hebben.

De inputs voor het model die leiden tot het voorspellen van een aanval zijn hier in het **rood** weergegeven.

(te vragen aan tijl A. is dit correct, is een ARP scan af te lezen van TCP traffiek?)

In [ ]:
y = dataset.y_train
attack_rows = y.index[y.label==1].tolist()
normal_rows = y.index[y.label==0].tolist()

shap.initjs()

#ARP scan
idx = attack_rows[5]
print(f"## row number: {idx}")
display(shap.force_plot(explainer.expected_value[1], values[1][idx,:], dataset.x_train.iloc[idx,:]))

### NMAP scan
Vervolgens worden alle poorten van een specifiek modbus device uitgelezen. Andere input waardes zijn van belang hier.

In [ ]:
#poorten modbus uitlezen
idx = attack_rows[90]
print(f"## row number: {idx}")
display(shap.force_plot(explainer.expected_value[1], values[1][idx,:], dataset.x_train.iloc[idx,:]))

### Aanpassen registers
Als laatste stap worden er een aantal registers overschreven door de aanvaller, opnieuw zien we dat andere input waardes van belang zijn hier.

In [ ]:
#data registers overschrijven
idx = attack_rows[400]
print(f"## row number: {idx}")
display(shap.force_plot(explainer.expected_value[1], values[1][idx,:], dataset.x_train.iloc[idx,:]))

### Gemiddeld belang van input

We kunnen kijken naar het gemiddeld belang van elke input over de hele dataset waar een ARP en NMAP scan plaatsvinden,\
alsook registers worden aangepast.

In [ ]:
shap.summary_plot(values[1], dataset.x_train, class_names=model.get_class_names(), plot_type="bar")

In [ ]:
#schrijven van holding registers
shap.force_plot(explainer.expected_value[1], values[1][3000:3500,:], dataset.x_train.iloc[3000:3500,:])

In [ ]:
#scan van registers
shap.force_plot(explainer.expected_value[1], values[1][2000:3000,:], dataset.x_train.iloc[2000:3000,:])

In [ ]:
#ARP scan en begin scan
shap.force_plot(explainer.expected_value[1], values[1][0:1100,:], dataset.x_train.iloc[0:1100,:])

In [ ]:
dataset.data.label.plot()
y = dataset.y_train
attack_rows = y.index[y.label==1].tolist()
normal_rows = y.index[y.label==0].tolist()

In [ ]:
pred_train, pred_test = model.predict()
x_train = dataset.x_train
x_test = dataset.x_test
mask = pred_train != dataset.y_train.label.values
# mask = pred_test != dataset.y_test.label.values
x_train[mask]

In [ ]:
x_train

In [ ]:
import pickle
data_dir = 'multirun/2023-01-16/11-21-04/9/NprobeData.pkl'

with open(data_dir,'rb') as f:
    dataset = pickle.load(f)


In [ ]:
dataset.y_train.value_counts()

In [ ]:
x_train, x_test = dataset.x_train, dataset.x_test
y_train, y_test = dataset.y_train, dataset.y_test

In [ ]:
import matplotlib.pyplot as plt

ecod = model.model
train_scores, test_scores = ecod.decision_function(x_train), ecod.decision_function(x_test)

print(ecod.threshold_)
plt.hist(train_scores, bins='auto') # arguments are passed to np.histogram
plt.title("Outlier score")
plt.show()



In [ ]:
from sklearn.metrics import classification_report

model.model.threshold_ = 19.5
pred_train, pred_test = model.predict()
print(classification_report(y_train, pred_train))
print(classification_report(y_test, pred_test))


In [ ]:
def descriptive_stat_threshold(df, pred_score, threshold):
    # Let's see how many '0's and '1's.
    df = pd.DataFrame(df)
    df['Anomaly_Score'] = pred_score
    df['Group'] = np.where(df['Anomaly_Score'] < threshold, 'Normal', 'Outlier')

    # Now let's show the summary statistics:
    cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
    cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
    stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
    stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
    return stat

In [ ]:
import numpy as np
descriptive_stat_threshold(x_train, train_scores, 19.5)

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()

In [ ]:
model.feature_importances_

In [ ]:
import os

os.chdir("multirun/2023-01-23/16-15-28/")
os.getcwd()

In [ ]:
_, x, _  = next(os.walk('.'))
x

In [ ]:
for file in os.listdir(x[0]):
    if file.endswith('.txt'):
        lines = open(f'{x[0]}/{file}').readlines()
        for line in lines:
            if 'macro avg' in line:
                print(line.split()[-2])
    

In [ ]:
                
                        

plot_general_graph('.')

In [ ]:
os.chdir('/home/aaron/projecten/ML-OT/multirun/2023-01-24/13-33-50')
os.getcwd()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

results = {'f1-macro':[0.97 ,0.93, 0.69, 0.71, 0.58, 0.58, 1.00, 0.98], 'set':["train","test"]*4, 
           'algo':["randomf","randomf","iforest","iforest", "iforest","iforest", "randomf", "randomf"],
          'dataset':["nprobe","nprobe","tshark","tshark", "nprobe","nprobe","tshark","tshark"]}
df = pd.DataFrame(results)

import os

def get_scores(path_run: str):
    scores, algo, ds = [], [], []
    
    #get local directories 1-level deep
    _, dirs, _ = next(os.walk(path_run))
    for d in dirs:
        for file in os.listdir(d):
            if file.endswith('.txt'):
                lines = open(f"{d}/{file}").readlines()
                for line in lines:
                    if 'macro avg' in line:
                        scores.append(float(line.split()[-2]))
                        algo.append(file[:-4].lower())
                        ds.append(d.split('dataset=')[1].split(',')[0])
                        
    results = {'F1-macro':scores,
              'Dataset':ds,
              'Algorithm': algo,
              'set':["train","test"]*int(len(scores)/2)}
    
    return pd.DataFrame(results)



def plot_overview(df: pd.DataFrame):
    f, axs = plt.subplots(1, len(df.Dataset.unique()), sharey=True)
    subplots = []
    for idx , dataset in enumerate(df.Dataset.unique()):
        subplots.append(sns.barplot(data=df[df.Dataset == dataset], x='Algorithm', y='F1-macro', hue='set', ax=axs[idx]))
        subplots[idx].legend_.remove()
        subplots[idx].set_title(dataset.upper())
        for container in subplots[idx].containers:
            subplots[idx].bar_label(container)
            
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
    for ax in axs:
        ax.tick_params(axis='x', rotation=90)
    
    f.tight_layout()
    plt.savefig('overview-scores.pdf')

In [ ]:
df = get_scores('.')
display(df)
plot_overview(df)

In [ ]:
import pickle
with open('model.pkl','rb') as f:
    model = pickle.load(f)

with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

model

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


#goeie grafiek als argument dat verkeer cyclisch is en afwijking hierop als anomaly kunnen gezien worden
ds = data.data.reset_index()
ds = ds[ds['ip.proto_tcp'] == 1]
ds = ds.rename(columns={"index":"packet no."})
display(ds)
plt.figure(figsize=(14,5))
sns.scatterplot(data= ds[ds['label']==1], x="packet no.", y="mean_packets", color="red", marker="X", label="aanval")
sns.lineplot(data = ds, x="packet no.", y="mean_packets", alpha=0.5, label="normale traffiek")
plt.title("Gemiddeld aantal paketten per minuut op netwerk")
plt.xlabel("Packet nummer")
plt.ylabel("Gemiddelde van paketten")
plt.savefig("cyclical.png")

In [ ]:
ds[ds['label']==1]


In [ ]:
from pyod.models.iforest import IForest

IForest?

# Machine Learning in Operational Technology (ML-OT)
ML-OT is a toolbox to apply different algorithms and analyses on network data.
1. Different options for experiments can be set with the commandline and easily reproduced. The Open source project Hydra from facebook research provides this functionality.
2. Datasets made by different parsers for network data are supported f.e. tshark, nProbe.
3. Automatic evaluation and analyses o.a. statistics, explainable AI, ..

## Installation

Install pytorch with instructions from official website https://pytorch.org/get-started/locally/
This depends on specific operating system, hardware and software.
```console
pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
```
Install remaining libraries of the requirements file.
```console
pip install -r requirements.txt
```
This command turns on tab completion for hydra.
```console
eval "$(python main.py -sc install=bash)"
```

When you make use of an virtual environment you can add this command to the activation script.
This will automatically activate tab completion when entering your virtual environment.
Use the absolute pad to the `main.py` script.
```console
echo 'eval "$(python <path_main_script> -sc install=bash)"' | tee -a env/bin/activate
```

## Usage

### Experiments with terminal
You can smoothly select different options for experiments and tab completion is enabled. 
This recording is a short showcase of some simple experiments.
[![asciicast](https://asciinema.org/a/Fa6K17UslsOcGqka3h8nmLak3.svg)](https://asciinema.org/a/Fa6K17UslsOcGqka3h8nmLak3)

Using this procedure you can explore the different configurations.
To apply one or more algorithms for one or more datasets add the `--multirun` arg to the command:
```console
python main.py --multirun dataset=tshark,nprobe model=randomforest,XGB
```
You can also do a **grid search** or sweep on a single algorithm:
```console
python main.py --multirun dataset=tshark model=randomforest model.n_estimators=2,5,10 model.max_depth=5,10,20
```
To perform a smarter hyperparameter search make use of `hp_search.py`:
```console
python hp_search.py --multirun dataset=tshark model=randomforest 'model.n_estimators=range(10,200)' 'model.max_depth=range(5,20)'
```
This will make use of the nevergrad plugin for hydra to search the parameter space and returns the parameter with the best average f1-score.
You can find more information on how to use nevergrad [here](https://hydra.cc/docs/plugins/nevergrad_sweeper/#defining-the-parameters)

### Experiments through config
Hydra uses a collection of config files in the background for the tab completion in terminal.
The config files can be adapted directly to have more extended control over the experiment.

The `configs` directory contains three important subdirectories:
1. `dataset`, contains the different options for datasets such as test size, data directory and whether labels should be binary or multi-class.
2. `model`, contains the different hyperparameters.
3. `analysis`, contains the different functions to apply after training for each model.

> Note that there are two types of models: classification and anomaly. Anomaly can only handle binary labels and classification can handle both binary and multi-class labels. 

To adapt the parameter search space used by the hp_search.py script adapt the config file `configs/hydra/sweeper/hyperparam_search.yaml`.
The default options here are for a randomforest model with the number of trees between 10 and 150. 
```yaml
parametrization:
    model : randomforest
    model.n_estimators:
        lower: 10
        upper: 150
        integer: True
```

You can also create a new config file, f.e. `my_search_space`, under the same directory and adapt the top level config file `configs/config_hpsearch.yaml` to:
```yaml
defaults:
    - _self_
    - dataset: tshark
    - model: randomforest
    - override hydra/sweeper: my_search_space  
```

This way you can maintain different experiments easily.

### Outputs

In the `outputs` directory you can find results of experiments with a single option of configs.
* main.log file containing all the executed steps.
* An evaluation for the model, report on the scores for the different classes.
* Analysis folder with different produced graphs.

These graphs are produced by different functions and they can be changed from the config files in directory `configs/analysis` each of the models contain different config files.

In the `multirun` directory you can find results of experiments with multiple options for config or sweeps. For grid search sweeps it will contain the same output as a normal experiment but in different subdirectories.For sweeps with nevergrad it will contain the different scores for each hyperparameter setting and the best found hyperparameters.


## Extendability
Datasets, models and analysis functions can be added to this project.

### Dataset
Dataset classes extend an interface.

```python
class InterfaceData(Dataset):
    def __init__(self, data_path: str, test_size: float, label_type: str, shuffle: bool):
        """Initialize dataset object with properties"""
        self.data_path = data_path
        self.test_size = test_size
        self.label_type = label_type
        self.shuffle = shuffle
        
        self.data=None
        self.x_train, self.x_test = None, None
        self.y_train, self.y_test = None, None
    
    def __len__(self):
        """method to retrieve length of data contained"""
        return len(self.data)
    
    def __getitem__(self, idx):
        """method to retrieve row of data"""
        return self.data.iloc[idx,:]           
    
    def load(self) -> None:
        """Load in dataset and store in object"""
        pass
    
    def preprocess(self) -> None:
        """Apply a specified preprocessing function and store in object"""
        pass
        
    def postprocess(self) -> None:
        """Any processing applied after splitting data, such as rescaling store in object"""
        pass
```

#### Step 1
To add a new dataset, extend the InterfaceData class in a python file under `src/datasets/`

#### Step 2
Write seperate functions for loading, preprocessing and postprocessing. You could also adapt the constructor, most of the time this will not be necessary.

#### Step 3
Add a new yaml file in the directory `configs/dataset`.\
It should target the new dataset class you created, a data path, test size, label_type and shuffle argument.
The tshark config looks like this:
```yaml
_target_: src.datasets.tshark.TsharkData
data_path: 'Data/tshark/scenario1'
label_type: 'binary'
test_size: 0.3
shuffle: False
```

### Model

Model classes extend an interface. 
```python
class InterfaceModel():
    def __init__(self, **kwargs):
        """ Initialize model_type as 'supervised','semi-supervised' or 'unsupervised'
        and pass kwargs as hyperparameters model"""
        self.model_type=None
        pass
    
    def fit(self, dataset: InterfaceData) -> None:
        """Apply model specific fit function"""
        pass
    
    def predict(self, dataset: InterfaceData) -> Union[tuple[DataFrame, DataFrame], DataFrame]:
        """Return one or two pandas dataframes with predictions"""
        pass
    
    def evaluate(self, dataset: InterfaceData, preds) -> None:
        """Calculate standard evaluation scores for model and save in outputs"""
        pass
    
    def score(self, dataset: InterfaceData) -> float:
        """Get singular score on relevant piece of data, depends of model type"""
        pass
    
    def get_class_names(self, dataset: InterfaceData) -> list:
        """Return original class names in list or [0,1] for binary datasets"""
        pass
```

To add a new model 
#### Step 1
Extend the InterfaceModel class in a python file under `src/models/`. 

#### Step 2
Implement the different functions and the constructor of your subclass. 

#### Step 3
Add a new yaml file in the directory `configs/model`. 
It should target the new model class you created and add the hyperparameters of the model.

The randomforest config looks like this:
```yaml
_target_: src.models.classification.RandomForest #name of class
verbose: 0
n_estimators: 10
criterion: "gini"
max_depth: null #null converted to None by Hydra
max_features: "sqrt"
max_leaf_nodes: null
n_jobs: null
class_weight: null
```

### Analysis functions
Analysis functions follow the same interface
```python
def new_analysis_func(dataset: InterfaceData, model: InterfaceModel, **kwargs):
    ...
```
Each analysis function should accept a dataset object, model object and any additional arguments that are needed.
To add a new function

#### Step 1
Define a new function using the interface described above. `src.explain` contains a number of analysis functions.

#### Step 2
Add the analysis function to the config of a model under `configs/analysis/<model_name>.yaml`.\
It should target your newly made analysis function with *\_target\_*.\
It should also contain a data and model argument with *???*. This means it will get filled in by your chosen dataset and model objects in the experiment. If there are any additional arguments expected by the function you can add them under model.

For example randomforest contains a function to explain feature importance with shap library.\
shaptotal can be any chosen name and will be displayed in the logs.
```yaml
shaptotal:
    _target_: src.explain.shap_importance
    dataset: ???
    model: ???
```


## Relevant Documentation

* Hydra
    - nevergrad: [link](https://hydra.cc/docs/plugins/nevergrad_sweeper/)
    - instantiate objects/call functions with hydra: [link](https://hydra.cc/docs/1.1/advanced/instantiate_objects/overview/)
* Scikit-Learn [link](https://scikit-learn.org/stable/)
* Python Outlier Detection (PyOD) [link](https://github.com/yzhao062/pyod)
* Extreme Gradient Boosting (XGB) [link](https://github.com/dmlc/xgboost/)
## References

**Datasets**
* nprobe UNSW'15 https://staff.itee.uq.edu.au/marius/NIDS_datasets/#RA4
* tshark lemay https://github.com/antoine-lemay/Modbus_dataset
* Proprietary Datasets generated for the GAICIA project, more information [link](https://gaicia.securityandprivacy.be/)

**Papers**
* [link](https://www.usenix.org/conference/cset16/workshop-program/presentation/lemay) Providing SCADA Network Data Sets for Intrusion Detection Research. (A. Lemay & J. M. Fernandez)
* [link](https://arxiv.org/abs/1905.11757) Evaluation of machine learning based anomaly detection algorithms on modbus/TCP data set. (SD.  Anton & S. Kanoor)
* [link](https://arxiv.org/abs/2011.09144) NetFlow Datasets for Machine Learning-based Network Intrusion Detection Systems. (M. Sarhan & S. Layeghy)

In [ ]:
!ls